In [0]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

nlp = spacy.load('en_core_web_lg')

In [0]:
from collections import defaultdict

In [0]:
df = pd.read_csv("/dbfs/FileStore/shared_uploads/t_karthik.ragunath@tatadigital.com/ner_datasetreference.csv", encoding='latin1')
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')
df = df.fillna('')
df.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [0]:
df.columns

Out[6]: Index(['Sentence #', 'Word', 'POS', 'Tag'], dtype='object')

In [0]:
df_grouped = df.groupby('Sentence #').agg(lambda x: ' '.join(x)).reset_index()

In [0]:
df_grouped.head(20)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP CC VB DT NN IN JJ NNS IN DT NN .,O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O
1,Sentence: 10,"Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .","JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT NN NNP , IN DT NNP NN NN VBZ VBG .",B-gpe O O O O O O O O O O O O O O B-tim O O O B-org O O O O O
2,Sentence: 100,"Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South ...","NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ NNP NNS VBP VBN TO VB VBN TO VB DT JJR JJ NN IN JJ NNP NNP .",O O B-tim O O O O O B-geo O O O O O B-org O O O O O O O O O O O O O O B-geo I-geo O
3,Sentence: 1000,They left after a tense hour-long standoff with riot police .,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O
4,Sentence: 10000,"U.N. relief coordinator Jan Egeland said Sunday , U.S. , Indonesian and Australian military helicopters are ferrying out food and supplies to remote areas of western Aceh province that ground crew...","NNP NN NN NNP NNP VBD NNP , NNP , JJ CC JJ JJ NNS VBP VBG RP NN CC NNS TO VB NNS IN JJ NNP NN IN NN NNS MD RB VB .",B-geo O O B-per I-per O B-tim O B-geo O B-gpe O B-gpe O O O O O O O O O O O O O B-geo O O O O O O O O
5,Sentence: 10001,"Mr. Egeland said the latest figures show 1.8 million people are in need of food assistance - with the need greatest in Indonesia , Sri Lanka , the Maldives and India .","NNP NNP VBD DT JJS NNS VBP CD CD NNS VBP IN NN IN NN NN : IN DT NN JJS IN NNP , NNP NNP , DT NNP CC NNP .",B-per I-per O O O O O O O O O O O O O O O O O O O O B-tim O B-per B-gpe O O B-geo O B-geo O
6,Sentence: 10002,He said last week 's tsunami and the massive underwater earthquake that triggered it has affected millions in Asia and Africa .,PRP VBD JJ NN POS NN CC DT JJ NN NN WDT VBD PRP VBZ VBN NNS IN NNP CC NNP .,O O O O O O O O O O O O O O O O O O B-geo O B-geo O
7,Sentence: 10003,"Some 1,27,000 people are known dead .",DT CD NNS VBP VBN JJ .,O O O O O O O
8,Sentence: 10004,"Aid is being rushed to the region , but the U.N. official stressed that bottlenecks and a lack of infrastructure remain a challenge .","NNP VBZ VBG VBN TO DT NN , CC DT NNP NN VBD IN NNS CC DT NN IN NN VBP DT NN .",O O O O O O O O O O B-geo O O O O O O O O O O O O O
9,Sentence: 10005,Lebanese politicians are condemning Friday 's bomb blast in a Christian neighborhood of Beirut as an attempt to sow sectarian strife in the formerly war-torn country .,JJ NNS VBP VBG NNP POS NN NN IN DT JJ NN IN NNP IN DT NN TO VB JJ NN IN DT RB JJ NN .,B-gpe O O O B-tim O O O O O O O O B-geo O O O O O O O O O O O O O


In [0]:
len(df_grouped)

Out[9]: 47959

Creating Gold Standard Data

In [0]:
ent_sentences = []
for sentence in df_grouped['Word']:
    doc = nlp(sentence)
    entities_dict = dict()
    entities_dict['entities'] = []
    for ent in doc.ents:
        entities_dict['entities'].append((ent.start_char, ent.end_char, ent.label_))
    ent_sentences.append((sentence, entities_dict))

In [0]:
ent_sentences[:10]

Out[28]: [('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
 {'entities': [(0, 9, 'CARDINAL'),
 (48, 54, 'GPE'),
 (77, 81, 'GPE'),
 (111, 118, 'NORP')]}),
 ('Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .',
 {'entities': [(0, 7, 'NORP'), (87, 96, 'DATE'), (108, 112, 'ORG')]}),
 ('Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .',
 {'entities': [(20, 28, 'DATE'),
 (62, 69, 'GPE'),
 (97, 104, 'ORG'),
 (190, 206, 'GPE')]}),
 ('They left after a tense hour-long standoff with riot police .',
 {'entities': [(24, 28, 'TIME')]}),
 ('U.N. relief coordinator Jan Egeland said Sunday , U.S. , Indonesian and Australian military helicopters are ferrying out food and supplies to remote areas of western Aceh province that ground crews can not reach .',
 {'entities': [(0, 4, 'ORG'),
 (24, 35, 'PERSON'),
 (41, 47, 'DATE'),
 (50, 54, 'GPE'),
 (57, 67, 'NORP'),
 (72, 82, 'NORP'),
 (166, 170, 'GPE')]}),
 ('Mr. Egeland said the latest figures show 1.8 million people are in need of food assistance - with the need greatest in Indonesia , Sri Lanka , the Maldives and India .',
 {'entities': [(4, 11, 'PERSON'),
 (41, 52, 'CARDINAL'),
 (119, 128, 'GPE'),
 (131, 140, 'GPE'),
 (147, 155, 'GPE'),
 (160, 165, 'GPE')]}),
 ("He said last week 's tsunami and the massive underwater earthquake that triggered it has affected millions in Asia and Africa .",
 {'entities': [(8, 20, 'DATE'),
 (98, 106, 'CARDINAL'),
 (110, 114, 'LOC'),
 (119, 125, 'LOC')]}),
 ('Some 1,27,000 people are known dead .',
 {'entities': [(5, 13, 'CARDINAL')]}),
 ('Aid is being rushed to the region , but the U.N. official stressed that bottlenecks and a lack of infrastructure remain a challenge .',
 {'entities': [(44, 48, 'ORG')]}),
 ("Lebanese politicians are condemning Friday 's bomb blast in a Christian neighborhood of Beirut as an attempt to sow sectarian strife in the formerly war-torn country .",
 {'entities': [(0, 8, 'NORP'),
 (36, 42, 'DATE'),
 (62, 71, 'NORP'),
 (88, 94, 'GPE')]})]

In [0]:
len(ent_sentences)

Out[29]: 47959

Fetching NER TAG to be added

In [0]:
color_dataset = pd.read_csv("/dbfs/FileStore/shared_uploads/t_karthik.ragunath@tatadigital.com/colours_rgb_shades.csv")
color_dataset['Color Name'] = color_dataset['Color Name'].fillna('')
color_names = []
for color in color_dataset['Color Name']:
    color_names.append(''.join(' ' + c if c.isupper() else c for c in color).lower().strip())
color_dataset['Color Names Cleaned'] = color_names
color_dataset['Color Names Cleaned'][:10]

Out[30]: 0 grey
1 grey, silver
2 grey
3 light gray
4 light slate grey
5 slate gray
6 slate gray1
7 slate gray2
8 slate gray3
9 slate gray4
Name: Color Names Cleaned, dtype: object

In [0]:
color_names_updated = []
for ind_color in color_names:
    comma_sep_colors = ind_color.split(',')
    for color in comma_sep_colors:
        color = ' '.join(color.split())
        color_names_updated.append(color)

In [0]:
color_names_set = set(color_names_updated)

In [0]:
color_names_set.remove('')

In [0]:
training_color_data = []
#actually annotaion must be done with context and not as empty words
for color_name in color_names_set:
  training_color_data.append((color_name, {"entities":[(0, len(color_name), "COLOR")]}))

In [0]:
training_color_data[:10]

Out[35]: [('seashell1', {'entities': [(0, 9, 'COLOR')]}),
 ('deep pink1', {'entities': [(0, 10, 'COLOR')]}),
 ('fuchsia', {'entities': [(0, 7, 'COLOR')]}),
 ('light salmon1', {'entities': [(0, 13, 'COLOR')]}),
 ('thistle4', {'entities': [(0, 8, 'COLOR')]}),
 ('goldenrod2', {'entities': [(0, 10, 'COLOR')]}),
 ('bronze i i', {'entities': [(0, 10, 'COLOR')]}),
 ('grey25', {'entities': [(0, 6, 'COLOR')]}),
 ('sienna2', {'entities': [(0, 7, 'COLOR')]}),
 ('salmon4', {'entities': [(0, 7, 'COLOR')]})]

In [0]:
complete_training_data = ent_sentences + training_color_data
complete_training_data[:10]

Out[37]: [('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
 {'entities': [(0, 9, 'CARDINAL'),
 (48, 54, 'GPE'),
 (77, 81, 'GPE'),
 (111, 118, 'NORP')]}),
 ('Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .',
 {'entities': [(0, 7, 'NORP'), (87, 96, 'DATE'), (108, 112, 'ORG')]}),
 ('Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .',
 {'entities': [(20, 28, 'DATE'),
 (62, 69, 'GPE'),
 (97, 104, 'ORG'),
 (190, 206, 'GPE')]}),
 ('They left after a tense hour-long standoff with riot police .',
 {'entities': [(24, 28, 'TIME')]}),
 ('U.N. relief coordinator Jan Egeland said Sunday , U.S. , Indonesian and Australian military helicopters are ferrying out food and supplies to remote areas of western Aceh province that ground crews can not reach .',
 {'entities': [(0, 4, 'ORG'),
 (24, 35, 'PERSON'),
 (41, 47, 'DATE'),
 (50, 54, 'GPE'),
 (57, 67, 'NORP'),
 (72, 82, 'NORP'),
 (166, 170, 'GPE')]}),
 ('Mr. Egeland said the latest figures show 1.8 million people are in need of food assistance - with the need greatest in Indonesia , Sri Lanka , the Maldives and India .',
 {'entities': [(4, 11, 'PERSON'),
 (41, 52, 'CARDINAL'),
 (119, 128, 'GPE'),
 (131, 140, 'GPE'),
 (147, 155, 'GPE'),
 (160, 165, 'GPE')]}),
 ("He said last week 's tsunami and the massive underwater earthquake that triggered it has affected millions in Asia and Africa .",
 {'entities': [(8, 20, 'DATE'),
 (98, 106, 'CARDINAL'),
 (110, 114, 'LOC'),
 (119, 125, 'LOC')]}),
 ('Some 1,27,000 people are known dead .',
 {'entities': [(5, 13, 'CARDINAL')]}),
 ('Aid is being rushed to the region , but the U.N. official stressed that bottlenecks and a lack of infrastructure remain a challenge .',
 {'entities': [(44, 48, 'ORG')]}),
 ("Lebanese politicians are condemning Friday 's bomb blast in a Christian neighborhood of Beirut as an attempt to sow sectarian strife in the formerly war-torn country .",
 {'entities': [(0, 8, 'NORP'),
 (36, 42, 'DATE'),
 (62, 71, 'NORP'),
 (88, 94, 'GPE')]})]

Before Training

In [0]:
nlp_update = spacy.load('en_core_web_lg')

In [0]:
example_doc = nlp('In USA I bought a red salwar and a blue kurta')
for ent in example_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

USA 3 6 GPE

In [0]:
import random
from spacy.training import Example
path_to_model = '/dbfs/mnt/nemo/'
batch_sizes = spacy.util.compounding(4.0, 32.0, 1.001)
optimizer = nlp_update.resume_training()
epochs = 20
for epoch in range(epochs):
    random.shuffle(complete_training_data)
    for batch in spacy.util.minibatch(complete_training_data, size=batch_sizes):
        examples=[]
        for text, annotations in batch:
            doc = nlp_update.make_doc(text)
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        losses = {}
        nlp_update.update(examples, sgd=optimizer, drop=0.35, losses=losses)
        print("Losses ({}/{})".format(epoch + 1, epochs), losses)

Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1.3264549930007499}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.10562578108671387}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1.5905878236412896}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.6597108539554648}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.201172102948186}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 4.000174385768407}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.2231895514083306}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.9095646022228583}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.6989216266421137}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.7315765020962486}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.1281059255934984}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.0578952511267453}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.5449396882275668}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1.5626279143844088}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.17978042929113025}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.333245601341906}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1.1484843907524602}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.5628610774574987}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.1696372296174617}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.8376048726771985}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.6445204033291505}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 5.278333569293623}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 5.0644843447493475}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.8146054156129103}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.451320071280036}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1.3400700260699696}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.407002796075572}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.0164756792952714}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.07025767459703387}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.17819813943288673}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.39246249691325497}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1.5898472154411318}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 4.561482327081945}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 6.9488607982131025}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.3791299954050498}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.203729844491683}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.27914366973885857}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.1282268119258925}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.8696383647118457}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.111123054054192}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.5311681329589026}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.5975862698578736}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.398183076964842}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.0005643307200267648}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 0.9528033402997064}
Losses (1/20) {'tok2vec': 0.0, 'tagger': 

In [0]:
example_doc = nlp_update('In USA I bought a red salwar and a blue kurta')
for ent in example_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

USA 3 6 GPE
blue kurta 35 45 COLOR

In [0]:
path_to_model = '/dbfs/mnt/nemo/ner_color_trained'
nlp_update.to_disk(path_to_model)

In [0]:
example_doc = nlp_update('red')
for ent in example_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

red 0 3 COLOR

In [0]:
example_doc = nlp('red')
for ent in example_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)